In [1]:
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


## Using APIs to source data from tamilnadu covid beds data

In [2]:
url = "https://mskian.github.io/tamil-nadu-covid-beds-api/api.json"
resp = requests.get(url)
if resp.status_code == 200:
    bed_data = resp.json()
else:
    print("Failed to fetch data:", resp.status_code)
    bed_data = {}

In [3]:
type((bed_data))

list

In [4]:
print(bed_data[0].keys())

dict_keys(['S', 'District', 'Institution', 'COVID_BEDS_Total', 'COVID_BEDS_Occupied', 'COVID_BEDS_Vacant', 'OXYGEN_SUPPORTED_BEDS_Total', 'OXYGEN_SUPPORTED_BEDS_Occupied', 'OXYGEN_SUPPORTED_BEDS_Vacant', 'NON_OXYGEN_SUPPORTED_BEDS_Total', 'NON_OXYGEN_SUPPORTED_BEDS_Occupied', 'NON_OXYGEN_SUPPORTED_BEDS_Vacant', 'ICU_BEDS_Total', 'ICU_BEDS_Occupied', 'ICU_BEDS_Vacant', 'VENTILATOR_Total', 'VENTILATOR_Occupied', 'VENTILATOR_Vacant', 'Last_updated', 'Contact_Number', 'Remarks'])


## Converting List of dictionaries into DataFrame

In [5]:
df = pd.DataFrame(bed_data)

In [6]:
df.head()

,S,District,Institution,COVID_BEDS_Total,COVID_BEDS_Occupied,COVID_BEDS_Vacant,OXYGEN_SUPPORTED_BEDS_Total,OXYGEN_SUPPORTED_BEDS_Occupied,OXYGEN_SUPPORTED_BEDS_Vacant,NON_OXYGEN_SUPPORTED_BEDS_Total,...,NON_OXYGEN_SUPPORTED_BEDS_Vacant,ICU_BEDS_Total,ICU_BEDS_Occupied,ICU_BEDS_Vacant,VENTILATOR_Total,VENTILATOR_Occupied,VENTILATOR_Vacant,Last_updated,Contact_Number,Remarks
0,{'NO': '1'},Ariyalur,A_S Hospital,30,29,1,8,8,0,12,...,0,10,9,1,2,2,0,2021_05_12 09:36:55,0,125_2021
1,{'NO': '2'},Ariyalur,Golden Hospital,31,30,1,23,23,0,3,...,1,5,5,0,5,5,0,2021_05_12 10:10:19,9487576493,Report 12/05/2021_
2,{'NO': '3'},Chengalpattu,Annai Arul Hospital,40,40,0,30,30,0,5,...,0,5,5,0,3,3,0,2021_05_12 10:35:13,4471700700,125_2021
3,{'NO': '4'},Chengalpattu,Chennai Emergency care Centre Pammal,16,16,0,11,11,0,4,...,0,1,1,0,1,1,0,2021_05_12 09:24:04,7358244662,125_2021
4,{'NO': '5'},Chengalpattu,Dr_ Rela Institute,160,160,0,130,130,0,0,...,0,30,30,0,10,10,0,2021_05_12 08:36:26,9600130888,12_05_21


In [7]:
df = df.drop('S', axis = 1)

In [8]:
df.head()

,District,Institution,COVID_BEDS_Total,COVID_BEDS_Occupied,COVID_BEDS_Vacant,OXYGEN_SUPPORTED_BEDS_Total,OXYGEN_SUPPORTED_BEDS_Occupied,OXYGEN_SUPPORTED_BEDS_Vacant,NON_OXYGEN_SUPPORTED_BEDS_Total,NON_OXYGEN_SUPPORTED_BEDS_Occupied,NON_OXYGEN_SUPPORTED_BEDS_Vacant,ICU_BEDS_Total,ICU_BEDS_Occupied,ICU_BEDS_Vacant,VENTILATOR_Total,VENTILATOR_Occupied,VENTILATOR_Vacant,Last_updated,Contact_Number,Remarks
0,Ariyalur,A_S Hospital,30,29,1,8,8,0,12,12,0,10,9,1,2,2,0,2021_05_12 09:36:55,0,125_2021
1,Ariyalur,Golden Hospital,31,30,1,23,23,0,3,2,1,5,5,0,5,5,0,2021_05_12 10:10:19,9487576493,Report 12/05/2021_
2,Chengalpattu,Annai Arul Hospital,40,40,0,30,30,0,5,5,0,5,5,0,3,3,0,2021_05_12 10:35:13,4471700700,125_2021
3,Chengalpattu,Chennai Emergency care Centre Pammal,16,16,0,11,11,0,4,4,0,1,1,0,1,1,0,2021_05_12 09:24:04,7358244662,125_2021
4,Chengalpattu,Dr_ Rela Institute,160,160,0,130,130,0,0,0,0,30,30,0,10,10,0,2021_05_12 08:36:26,9600130888,12_05_21


In [9]:
df.shape

(567, 20)

## Taking only numerical columns as they provide data about beds available

In [10]:
num_cols = ['COVID_BEDS_Vacant', 'ICU_BEDS_Vacant', 'VENTILATOR_Vacant', 'OXYGEN_SUPPORTED_BEDS_Vacant']
df[num_cols] =df[num_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

In [11]:
df.columns =df.columns.str.strip()

In [12]:
df[num_cols].columns

Index(['COVID_BEDS_Vacant', 'ICU_BEDS_Vacant', 'VENTILATOR_Vacant',
       'OXYGEN_SUPPORTED_BEDS_Vacant'],
      dtype='object')

## Normalize the data

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(df[num_cols])

## K-Nearest Neighbor Model

In [14]:
model = NearestNeighbors(n_neighbors = 5, metric = 'cosine')
model.fit(X)

NearestNeighbors(metric='cosine')

## Defining a function to recommend hospital 

In [15]:
import numpy as np
def recommend_similar_hospitals(hospital_name):
    mask = df['Institution'].str.contains(hospital_name, case = False, na = False)
    if not mask.any():
        return f"Hospital '{hospital_name} Not Found."
    idx = np.where(mask)[0][0]

    hospital_vector = X[idx].reshape(1,-1)
    distances, indices = model.kneighbors(hospital_vector
                                         )

    similar = df.iloc[indices[0]][['Institution', 'District', 'COVID_BEDS_Vacant', 'ICU_BEDS_Vacant', 'VENTILATOR_Vacant']]
    return similar

In [16]:
recommend_similar_hospitals("Golden Hospital")

,Institution,District,COVID_BEDS_Vacant,ICU_BEDS_Vacant,VENTILATOR_Vacant
462,"Aditya Kamaraja Hospital Pvt Ltd,Virugambakkam",Chennai,1,0,0
560,"Vasantha Hospital, Thiruvottiyur",Chennai,1,0,0
313,Shifa Hospital,Tirunelveli,1,0,0
266,"Saraswathy Multi Speciality Hospitals, Madipakkam",Chennai,1,0,0
1,Golden Hospital,Ariyalur,1,0,0
